In [15]:
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from openpyxl import load_workbook
import os
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import traceback
df = pd.read_excel("C://Users//chris//S&G wear Together GmbH//Bestellfenster - Dokumente//01 AUFTRÄGE//Schulen 2021,22//127_Sophie_Scholl_Realschule//5 Bestellung//bestellliste2022-02-21-21-29-39.xlsx")
df["Klasse"] = df["Product Variation"].str.split(
    "|", n=4, expand=True)[2].str.replace("Klasse:", "")
df.rename(columns={"Item Name(löschen)": "Produktname",
                   "Anzahl ": "Anzahl"}, inplace=True)

pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', 100)
df = pd.DataFrame(df.values.repeat(df.Anzahl, axis=0), columns=df.columns)
df.drop(['Anzahl', 'Product Variation', 'Bestellnotiz',
         'Bestellung Gesamtsumme(löschen)'], axis=1, inplace=True)
t = pd.CategoricalDtype(
    categories=['XS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL'], ordered=True)
df['Größe'] = pd.Series(df.Größe, dtype=t)
df.sort_values(by=['Klasse', 'Produktname', 'Farbe',
                   'Größe'], inplace=True, ignore_index=True)
# =WENN(UND(I2="Ja";J2="");D2;WENN(I2="Nein";"";WENNFEHLER(RECHTS(J2;LÄNGE(J2)-50);"")))
df["Individualisierungstext(zählt nur wenn Individualisierung Ja)"] = df.apply(
    lambda x: x['Input Fields'] if x['Individualisierung'] == 'Ja' else "", axis=1)
#df["Individualisierungstext(zählt nur wenn Individualisierung Ja)"] = np.where((~df['Input Fields'].isnull()) & (~df['Individualisierung']== 'Ja') ,df['Nachnahme (Rechnungsadresse)'],"")
df["Individualisierungstext(zählt nur wenn Individualisierung Ja)"] = df[
    "Individualisierungstext(zählt nur wenn Individualisierung Ja)"].str[50:]
df["Individualisierungstext(zählt nur wenn Individualisierung Ja)"] = df.apply(lambda x: x['Nachnahme (Rechnungsadresse)'] if pd.isnull(
    x['Individualisierungstext(zählt nur wenn Individualisierung Ja)']) else x['Individualisierungstext(zählt nur wenn Individualisierung Ja)'], axis=1)
df["Karton"] = (df.index / 20 + 1).astype(int)
df.drop(['Input Fields'], axis=1, inplace=True)

df.sort_values(by=['Karton', 'Klasse', 'Produktname',
                   'Farbe', 'Größe'], inplace=True, ignore_index=True)
df['Checkbox'] = '☐'
df['Unterschrift'] = ' '

df["Anzahl"] = 1
# df2= df2.pivot_table(index=['Produktname','Größe','Farbe'],
# columns='Individualisierung',
# margins = True,
# aggfunc='size',
# fill_value=0)

#wb = load_workbook(filename = 'vorlage_bestellliste_shop.xltx')
#ws = wb["Orders"]
#bestellungen = ws.tables["Bestellungen"]
# print(bestellungen)

df.columns = df.columns.astype(str)
print("pivot")
pivottableastable = df.pivot_table(
    index=["Produktname","Farbe","Größe"], values=["Anzahl","Individualisierung"], aggfunc={'Anzahl':len,'Individualisierung':(lambda x:(x=='Ja').sum())}, margins=True, margins_name='Grand Totals')

pivottableastable = pivottableastable.rename(columns={'Individualisierung': 'Anzahl Personalisierungen'})
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):    
    print(pivottableastable)
pivottable = pd.DataFrame(pivottableastable.to_records())
key = {"Schulpullover": "JH001",
       "Schulshirt": "B&C001",
       }
pivottable['Produktname-Lieferant']=pivottable['Produktname']
pivottable.replace({"Produktname-Lieferant": key}, regex=True, inplace = True)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):    
    print(pivottable)

pivot
                                                         Anzahl  \
Produktname                            Farbe      Größe           
Sophie Scholl Realschule Schulpullover Dunkelgrau XS         10   
                                                  S          10   
                                                  M           6   
                                                  L           4   
                                                  XXL         1   
                                       Navy       XS          6   
                                                  S          14   
                                                  M          22   
                                                  L          10   
                                                  XL          3   
Grand Totals                                                 86   

                                                         Anzahl Personalisierungen  
Produktname                          